## Création d'un utilisateur

Créer un utilisateur et les clés nécessaire au bon fonctionnement du chiffrement

In [98]:
from key_management.user_enrollment import enroll_user

if __name__ == "__main__":
    username = input("Entrez le nom d'utilisateur pour l'enrôlement : ")
    enroll_user(username)

Utilisateur 'Moi' enrôlé avec succès. Les clés sont stockées dans './users/Moi'


## Création des S-boxes

Afin de faire les permutations nécessaires dans COBRA, nous devons créer des S-boxes afin de permettre de mélanger les caractères de notre par un autre

In [6]:
from pathlib import Path
from outils.sboxes import generate_multiple_s_boxes, generate_inverse_s_boxes
from storage.sboxes_storage import save_s_boxes_to_file

PATH = Path("encryption")


if __name__ == "__main__":
    # Génération de 4 S-Boxes de taille 16 (pour blocs de 4 bits)
    s_boxes = generate_multiple_s_boxes(num_s_boxes=4, size=16)

    # Affichage des S-Boxes
    for i, s_box in enumerate(s_boxes):
        print(f"S-Box {i+1} : {s_box}")

    save_s_boxes_to_file(s_boxes, filename=PATH / "s_boxes.json")

    # Génération des S-Boxes inverses
    inverted_s_boxes = generate_inverse_s_boxes(s_boxes)

    # Affichage des S-Boxes inverses
    for i, inv_s_box in enumerate(inverted_s_boxes):
        print(f"S-Box inverse {i+1} : {inv_s_box}")

    save_s_boxes_to_file(inverted_s_boxes, filename=PATH /"inverted_s_boxes.json")
    

S-Box 1 : [0, 6, 15, 13, 14, 10, 12, 1, 2, 4, 7, 9, 5, 3, 8, 11]
S-Box 2 : [15, 8, 7, 1, 0, 2, 4, 12, 10, 13, 3, 9, 14, 11, 5, 6]
S-Box 3 : [6, 7, 2, 13, 3, 0, 15, 4, 9, 10, 12, 8, 11, 1, 5, 14]
S-Box 4 : [10, 4, 0, 3, 9, 12, 13, 6, 7, 1, 14, 5, 2, 15, 11, 8]
S-Boxes sauvegardées dans le fichier : encryption\s_boxes.json
S-Box inverse 1 : [0, 7, 8, 13, 9, 12, 1, 10, 14, 11, 5, 15, 6, 3, 4, 2]
S-Box inverse 2 : [4, 3, 5, 10, 6, 14, 15, 2, 1, 11, 8, 13, 7, 9, 12, 0]
S-Box inverse 3 : [5, 13, 2, 4, 7, 14, 0, 1, 11, 8, 9, 12, 10, 3, 15, 6]
S-Box inverse 4 : [2, 9, 12, 3, 1, 11, 7, 8, 15, 4, 0, 14, 5, 6, 10, 13]
S-Boxes sauvegardées dans le fichier : encryption\inverted_s_boxes.json


## Fonctions Présentent dans encryption/cobra_cipher.py

Test des fonctions dans cobra cipher

## XOR

In [ ]:
from encryption.xor import *

# Exemple d'utilisation
message = "Hello, World! This is a test message."
key = "Monmdp1234"

# Chiffrer le message
encrypted_message = encrypt_message_with_key(message, key)
print("Message chiffré (bytes):", encrypted_message)

# Déchiffrer le message
decrypted_message = decrypt_message_with_key(encrypted_message, key)
print("Message déchiffré:", decrypted_message)

Message chiffré (bytes): [  5  10   2   1  11  92  17 101  92  70  33  11  79  77  48  24  36  28
  78   4  23  80  80  18  71  81  62  27  78   0   1   3  62  14   9   8
  68  22  94  64  19 103  40  29  30   8  10   4 109  10   0  14  22   9
  65  70  90  91  35  65 106 105  96 116]
Message déchiffré: Hello, World! This is a test message for Serpent encryption.


## Permutation

In [ ]:
import random
from encryption.substitution import *

S-Boxes chargées depuis le fichier : encryption\s_boxes.json
S-Boxes chargées depuis le fichier : encryption\inverted_s_boxes.json
Blocs binaires : [4, 8, 6, 5, 6, 12, 6, 12, 6, 15, 2, 12, 2, 0, 5, 7, 6, 15, 7, 2, 6, 12, 6, 4, 2, 1, 2, 0, 5, 4, 6, 8, 6, 9, 7, 3, 2, 0, 6, 9, 7, 3, 2, 0, 6, 1, 2, 0, 7, 4, 6, 5, 7, 3, 7, 4, 2, 0, 6, 13, 6, 5, 7, 3, 7, 3, 6, 1, 6, 7, 6, 5, 2, 14]
Blocs chiffrés : [5, 10, 15, 3, 15, 1, 15, 1, 15, 0, 11, 1, 11, 13, 3, 9, 15, 0, 9, 11, 15, 1, 15, 5, 11, 4, 11, 13, 3, 5, 15, 10, 15, 7, 9, 2, 11, 13, 15, 7, 9, 2, 11, 13, 15, 4, 11, 13, 9, 5, 15, 3, 9, 2, 9, 5, 11, 13, 15, 8, 15, 3, 9, 2, 9, 2, 15, 4, 15, 9, 15, 3, 11, 14]
Message déchiffré : Hello, World! This is a test message.


## Combinaisons des Fonctions

Combinaisons des fonctions comme si nous voulons chiffrer un message

S-Boxes chargées depuis le fichier : encryption\s_boxes.json
S-Boxes chargées depuis le fichier : encryption\inverted_s_boxes.json
Message original : Hello, World! This is a test message.
encrypted_text : 
\e\Fld! Th$NPPGQst mes>J{:98?
Blocs chiffrés : [0, 6, 7, 1, 7, 2, 7, 2, 7, 13, 9, 2, 9, 14, 1, 5, 7, 13, 5, 9, 7, 2, 7, 0, 9, 15, 9, 14, 1, 0, 7, 6, 7, 8, 5, 12, 9, 14, 7, 8, 5, 12, 9, 14, 7, 15, 9, 14, 5, 0, 7, 1, 5, 12, 5, 0, 9, 14, 7, 3, 7, 1, 5, 12, 5, 12, 7, 15, 7, 5, 7, 1, 9, 10]

###########################

ús~ÄGv~ÏK~K~~OKO~KKx
